# Computer Vision Day 1 - Dense networks
We will train a dense network to classify CIFAR10 images. This code was inspired by the [TensorFlow 2 quickstart for beginners](https://www.tensorflow.org/tutorials/quickstart/beginner).

### How to use it?
The easiest way is to use Google Colab
* Go to https://colab.research.google.com
* Select `File > open notebook` and upload the notebook
* If you want to use a GPU go to `Runtime > Change runtime type`
* Start!

Or just press the button below and get started!

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/Digital-Health-UMCU/tensorflow_demo/1_intro.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/Digital-Health-UMCU/tensorflow_demo/1_intro.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

In [ ]:
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")

Load and prepare the [cifar10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset. Convert the samples from integers to floating-point numbers.

**Questions**
* Why do we divide by 255?
* Can you explain the shape?

In [ ]:
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(x_train.shape)

## Create a model
It is easiest to use a softmax activation in the last layer for multiclass classification problems. However, while using the softmax activition makes the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models whenusing a softmax output. 
Alternatively you can use a linear activation in the last layer and then apply `tf.nn.softmax(...)` on the predictions.

In [ ]:
model = tf.keras.models.Sequential([
  ...
])

In [106]:
#@title Example Model
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

# # Example convolutional neural network.
# model = tf.keras.models.Sequential([
#   tf.keras.layers.InputLayer(input_shape=(32,32,3)),
#   tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
#   tf.keras.layers.MaxPooling2D((2,2)),
#   tf.keras.layers.Dropout(0.2),
#   tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
#   tf.keras.layers.MaxPooling2D((2,2)),
#   tf.keras.layers.Dropout(0.2),
#   tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
#   tf.keras.layers.MaxPooling2D((2,2)),
#   tf.keras.layers.Dropout(0.2),
#   tf.keras.layers.GlobalAveragePooling2D(),
#   tf.keras.layers.Dense(10, activation='softmax')
# ])

Make some example predictions

In [ ]:
predictions = model(x_train[:1]).numpy()
predictions

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
model.summary()

In [ ]:
print(f"Loss on the train set: {loss_fn(y_train[:1], predictions).numpy():.2f}")
print(f"Accuracy on the train set: {model.evaluate(x_train,  y_train, verbose=0)[1]:.2f}")


Next, fit the model parameters using the `model.fit` method.

In [ ]:
model.fit(..., 
          ..., 
          epochs=...
          )

In [105]:
#@title Example answer
# model.fit(x_train, y_train, 
#           epochs=20, 
#           validation_split=0.2,
#           callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1),])

The `Model.evaluate` method checks the models performance.

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

## Look at some predictions
Class names for cifar10 can be found [here](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/cifar10/load_data).

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
# Get predictions
preds_test = model.predict(x_test).argmax(axis=1)

# define class names
class_names = {0: 'airplane',
               1:	'automobile',
               2:	'bird',
               3:	'cat',
               4:	'deer',
               5:	'dog',
               6:	'frog',
               7:	'horse',
               8:	'ship',
               9:	'truck'}

Look at some examples and plot the confusion matrix.

In [ ]:
index = 16
plt.title(f"Prediction: {class_names[preds_test[index]]}; True label: {class_names[y_test.ravel()[index]]}")
plt.imshow(x_test[index])

In [ ]:
keys = [k for k in class_names.keys()]
names = [v for v in class_names.values()]
cm = confusion_matrix(y_test.ravel(), preds_test, labels = keys)

plt.figure(figsize=(10, 10))
ax = plt.subplot()
ConfusionMatrixDisplay(cm, display_labels=names).plot(values_format='d', ax=ax)
plt.show()